## 推移確率行列の作成関数

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
def getCSV(file):
  return pd.read_csv(file, engine='python', encoding='utf-8')

In [4]:
def saveCSV(df, fname):
  pdf = pd.DataFrame(df)
  pdf.to_csv(fname, index=False)

In [5]:
popularity = getCSV('/content/drive/MyDrive/研究/BCMP/TransitionProbability/csv/popularity2.csv')
popularity2 = popularity.iloc[:,2:4].values.tolist()
len(popularity2[0])

2

In [6]:
distance = getCSV('/content/drive/MyDrive/研究/BCMP/TransitionProbability/csv/distance.csv')
distance

,fromid,toid,distance
0,1,2,274.600697
1,1,3,133.583301
2,1,4,290.322684
3,1,5,445.825595
4,1,6,468.562105
...,...,...,...
10327,39,41,109.152687
10328,39,42,96.459463
10329,40,41,162.517207
10330,40,42,79.469642


In [7]:
#利用するノード
node = list(popularity['Id'])
node.index(10)

1

In [8]:
#距離行列の作成
distance_matrix = np.zeros((len(node),len(node)))
for row in distance.itertuples(): #右三角行列で作成される
  if row.fromid in node and row.toid in node:
    distance_matrix[node.index(int(row.fromid))][node.index(int(row.toid))] = row.distance
for i in range(len(distance_matrix)): #下三角に値を入れる(対象)
  for j in range(i+1, len(distance_matrix)):
    distance_matrix[j][i] = distance_matrix[i][j]
distance_matrix

array([[   0.        ,  158.5314119 ,  163.3324365 , ..., 1110.882796  ,
         948.5230385 , 1109.261248  ],
       [ 158.5314119 ,    0.        ,  109.2007727 , ...,  984.4369174 ,
         822.1707311 ,  975.9158655 ],
       [ 163.3324365 ,  109.2007727 ,    0.        , ...,  948.3003713 ,
         786.0895554 ,  948.3764588 ],
       ...,
       [1110.882796  ,  984.4369174 ,  948.3003713 , ...,    0.        ,
         162.517207  ,   79.46964222],
       [ 948.5230385 ,  822.1707311 ,  786.0895554 , ...,  162.517207  ,
           0.        ,  173.86531   ],
       [1109.261248  ,  975.9158655 ,  948.3764588 , ...,   79.46964222,
         173.86531   ,    0.        ]])

In [9]:
#重力モデルで推移確率行列を作成 
def getGravity(distance, popularity): #distanceは距離行列、popularityはクラス分の人気度
  C = 0.1475
  alpha = 1.0
  beta = 1.0
  eta = 0.5
  class_number = len(popularity[0]) #クラス数
  tp = np.zeros((len(distance) * class_number, len(distance) * class_number))
  for r in range(class_number):
    for i in range(len(distance) * r, len(distance) * (r+1)):
      for j in range(len(distance) * r, len(distance) * (r+1)):
        if distance[i % len(distance)][j % len(distance)] > 0:
          tp[i][j] = C * (popularity[i % len(distance)][r]**alpha) * (popularity[j % len(distance)][r]**beta) / (distance[i % len(distance)][j % len(distance)]**eta)
  row_sum = np.sum(tp, axis=1) #行和を算出
  for i in range(len(tp)): #行和を1にする
    if row_sum[i] > 0:
      tp[i] /= row_sum[i]
  return tp 

In [14]:
p = getGravity(distance_matrix, popularity2)
#saveCSV(transitionprob, '/content/drive/MyDrive/研究/BCMP/TransitionProbability/csv/transition_test.csv')

In [12]:
#MVAファイルの取り込み
import sys
sys.path.append('/content/drive/MyDrive/研究/BCMP/')
import BCMP_MVA as mdl

In [18]:
#MVAの実施
import time
N = 33 #33
R = 2
K_total = 500
K = [(K_total + i) // R for i in range(R)] #クラス人数を自動的に配分する
mu = np.full((R, N), 1) #サービス率を同じ値で生成(サービス率は調整が必要)
type_list = np.full(N, 1) #サービスタイプはFCFS
m = np.full(N, 1) #今回は窓口数1(複数窓口は実装できていない)
bcmp = mdl.BCMP_MVA(N, R, K, mu, type_list, p, m)
start = time.time()
L = bcmp.getMVA()
elapsed_time = time.time() - start
print ("calclation_time:{0}".format(elapsed_time) + "[sec]")
print('L = \n{0}'.format(L))

calclation_time:331.1964535713196[sec]
L = 
[[  0.68714441   1.30866578]
 [  0.52863235   0.52773361]
 [  0.6666094    0.66486651]
 [  0.71189491   0.71219735]
 [  0.72789044   0.71658789]
 [  0.87207858   0.87319703]
 [  8.13399704  15.41765413]
 [ 11.24206377   5.92617826]
 [  4.53283857   8.60941224]
 [  0.91279238   0.88957988]
 [ 55.888067   103.36792579]
 [  1.01552042   0.98773331]
 [  0.89586644   0.86464189]
 [ 73.78108203  39.77871812]
 [  1.08898826   1.08176985]
 [  1.04939497   1.03250438]
 [ 11.96417802  21.51377528]
 [  3.71564013   6.91677122]
 [ 53.3187179   26.64472621]
 [  1.06912521   0.99258841]
 [  0.86612971   0.82835217]
 [  1.0904636    1.0149013 ]
 [  0.73633205   0.67553353]
 [  0.5889714    0.53952228]
 [  5.17326224   2.43758578]
 [  0.79621544   0.72473369]
 [  0.74624096   0.65408399]
 [  0.61150973   0.53783229]
 [  0.58836596   0.52700419]
 [  0.61172486   0.52827275]
 [  1.88853302   0.87960613]
 [  3.01459117   1.39952824]
 [  0.48513762   0.42581652]